In [11]:
import planetspy as ps
import planetspy.engines as eng
import numpy as np

bodies = [
    np.array([
        [0, 0, 0, 0, 0, 0, 6.371*10**6, 5.972*10**24],
        [6.371*10**6 + 400*10**3, 0, 0, 0, 7.6 *10**3, 100*10**3, 0]
    ]), [["Earth"], ["ISS"]]
]

sim = ps.Simulate(eng.PP, 100, 0.1, bodies, bailout = 90*3600/0.1,)

TypeError: Simulate() takes no arguments